In [1]:
import pandas as pd
import numpy as np

In [2]:
min_support = 0.2
input_fileName = 'sample.txt'
output_fileName = 'sample_output.txt'

In [3]:
df = pd.read_csv(input_fileName, sep=' ', header=None)
df.columns=['tx']

In [4]:
min_support = min_support*len(df)

In [54]:
class node:
  def __init__(self, item, frequent, parent, child=[]):
      self.item = item
      self.frequent = frequent
      self.parent = parent
      self.child = child
  
  def list_child_item(self):
    child_item_list=[]
    
    for child in self.child:
      child_item_list.append(child.item)
      
    return child_item_list
  
  def search_child(self,item):           
    for child in self.child:
      if item == child.item:
        return child
    return None
        
  def printTree(self,level=0):
    print('\t' * level, '[level: ', level, 'item: ', self.item, 'fre: ', self.frequent, ']')
    for child in self.child:
      child.printTree(level+1)
  
  def check_fre(self,discharge_list):
    
    if self.item in discharge_list:
      for child in self.child:
        child.parent = self.parent
        self.parent.child.append(child)
      self.parent.child.remove(self)
      self.parent.combine_common_child()
      self.child = []
  
  def recur_check_fre(self,discharge_list):
    self.printTree()
    self.check_fre(discharge_list)
    for child in self.child:
      child.recur_check_fre(discharge_list)
          
  def find_common_child(self):
    
    uniq, inv = np.unique(self.list_child_item, return_inverse=True)
    
    common_node_index = [[uniq[i], np.where(inv == i)[0]] for i in range(uniq.size)]
    
    return common_node_index
  
  def combine_common_child(self):

    common_list = self.find_common_child()
    
    for common in common_list:
      if(len(common[1])>1):
        
        index_list = list(common[1])
        
        new_child  = node(common[0],0,self,[])
        
        for index in index_list:
          child_node = self.child[index]
          new_child.frequent += child_node.frequent
          new_child.child.extend(child_node.child)
          
        new_child.combine_coomon_child()

        for index in sorted(index_list, reverse=True):
          del self.child[index]
        
        self.child.append(new_child)
    


In [7]:
# construct 1-itemset

itemset_1 = pd.DataFrame(columns=['item','frequent','head_list'])

for index,row in df.iterrows():
    
    tx = row['tx'].split(",")
    
    for item in tx:
        
        if item in itemset_1['item'].values:           
            itemset_1.loc[itemset_1['item']==item, 'frequent'] +=1
        else:
            itemset_1.loc[len(itemset_1.index)] = [item, 1,[]]
            

In [8]:
#sort by frequent

itemset_1 = itemset_1.sort_values('frequent',ascending=False)
itemset_1 = itemset_1.set_index('item')

#去掉小於min support

for i in range(0,len(itemset_1)): 
    
    if itemset_1.iloc[i].frequent < min_support:
        itemset_1 = itemset_1.iloc[0:i]
        break

itemset_1

,frequent,head_list
item,,
0,13,[]
9,11,[]
10,10,[]
1,7,[]
8,7,[]
7,7,[]
5,6,[]
4,6,[]
2,6,[]


In [9]:
#依據 itemset_1 排序 tx

sorted_item = itemset_1.index

for index,row in df.iterrows():
    
    tx = row['tx'].split(",")
    
    sorted_tx = []
    
    for item in sorted_item:
        
        if item in tx:
            sorted_tx.append(item)
    
    row['tx'] = sorted_tx
    


In [10]:
df

,tx
0,"[9, 10, 5]"
1,"[0, 10, 1, 8, 4, 6]"
2,"[0, 10, 1]"
3,[5]
4,"[0, 10, 1, 8, 3]"
5,[9]
6,"[0, 9, 5, 4, 2, 6]"
7,[3]
8,"[0, 9, 10, 7, 4, 6]"
9,"[0, 10, 8, 6]"


In [11]:
#construct tree

root = node('root',None,None,[])
root.printTree()

 [level:  0 item:  root fre:  None ]


In [12]:
for tx in df['tx']:
    
    search_node = root

    for item in tx:
        # 在目前search_node的child node有此item就+1
        # 沒有就新增 childe的node
        
        child_node = search_node.search_child(item)
            
        if(child_node != None):
            child_node.frequent +=1
        else:
            # new node
            child_node = node(item,1,search_node,[])
            # connect to header list
            itemset_1.loc[item].head_list.append(child_node)
            # connect to tree
            search_node.child.append(child_node)
                
        search_node = child_node

In [13]:
root.printTree()

 [level:  0 item:  root fre:  None ]
	 [level:  1 item:  9 fre:  3 ]
		 [level:  2 item:  10 fre:  2 ]
			 [level:  3 item:  5 fre:  1 ]
			 [level:  3 item:  1 fre:  1 ]
				 [level:  4 item:  7 fre:  1 ]
					 [level:  5 item:  5 fre:  1 ]
						 [level:  6 item:  2 fre:  1 ]
							 [level:  7 item:  3 fre:  1 ]
	 [level:  1 item:  0 fre:  13 ]
		 [level:  2 item:  10 fre:  4 ]
			 [level:  3 item:  1 fre:  3 ]
				 [level:  4 item:  8 fre:  2 ]
					 [level:  5 item:  4 fre:  1 ]
						 [level:  6 item:  6 fre:  1 ]
					 [level:  5 item:  3 fre:  1 ]
			 [level:  3 item:  8 fre:  1 ]
				 [level:  4 item:  6 fre:  1 ]
		 [level:  2 item:  9 fre:  8 ]
			 [level:  3 item:  5 fre:  1 ]
				 [level:  4 item:  4 fre:  1 ]
					 [level:  5 item:  2 fre:  1 ]
						 [level:  6 item:  6 fre:  1 ]
			 [level:  3 item:  10 fre:  3 ]
				 [level:  4 item:  7 fre:  1 ]
					 [level:  5 item:  4 fre:  1 ]
						 [level:  6 item:  6 fre:  1 ]
				 [level:  4 item:  1 fre:  2 ]
					 [level:  5

In [169]:
# Construct Mining FP Tree
pattern = []

for mining in itemset_1.index:

    sub_root = node('root',1,None,[])
    total_frequent ={}
    path_and_fre =[]
    sub_header =[]
    for mining_node in itemset_1.loc[mining].head_list:
        # construct a subtree which leaf is mining
        
        # get list from leaf to node
        
        path =[]
        mining_node_frequent = mining_node.frequent
        path.append(mining_node.item)
        
    
        mining_node_parent = mining_node.parent 
        
        while mining_node_parent.item != 'root':
            
            path.append(mining_node_parent.item)
            
            if mining_node_parent.item in total_frequent:
                total_frequent[mining_node_parent.item] += mining_node_frequent
            else:
                total_frequent[mining_node_parent.item] =1
                
            mining_node_parent = mining_node_parent.parent
        
        path.reverse()
        path_and_fre.append([path,mining_node_frequent])

    discharge_node = [k for k, v in total_frequent.items() if v < min_support]

    conditional_fp_tree = {k:v for k, v in total_frequent.items() if v >= min_support}
    # print(path_and_fre)
    # for path in path_and_fre:
    #     for item in path[0]:
    #         if item not in discharge_node:
    #             path[0].remove(item)
    # print(path_and_fre)

    # add node to subtree
        
    for pair in path_and_fre:
        
        search_node = sub_root
        
        path = pair[0]
        fre = pair[1]
        
        for item in path:
            
            if item not in discharge_node:
                child_node = search_node.search_child(item)
                
                if(child_node != None):
                    child_node.frequent +=mining_node_frequent
                else:
                    child_node = node(item,fre,search_node,[])
                    if item == mining:
                        sub_header.append(child_node)  
                    search_node.child.append(child_node)
                
                search_node = child_node



    for header in sub_header:
        path = []
        
        search_node = header
        fre = header.frequent
        
        while search_node.item != 'root':
            path.append(search_node.item)
            search_node = search_node.parent
            
        path.sort()
        pattern.append([path,fre/len(df)])
    


    for pa in pattern:
        search_pattern = pattern.copy()
        search_pattern.remove(pa)
        for pa_2 in search_pattern:
            if set(pa[0]).issubset(pa_2[0]):
                pa[1]+=pa_2[1]

    



 [level:  0 item:  root fre:  1 ]
	 [level:  1 item:  0 fre:  5 ]
		 [level:  2 item:  9 fre:  4 ]
			 [level:  3 item:  2 fre:  4 ]
		 [level:  2 item:  2 fre:  1 ]
	 [level:  1 item:  9 fre:  1 ]
		 [level:  2 item:  2 fre:  1 ]


In [170]:
with open(output_fileName, 'w') as f:
    for pa in pattern:
        f.write(pa[0],':',pa[1],'\n')



{'9': 5, '0': 5}